osconfeed.py：下载 osconfeed.json

In [4]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg) #1
        with urlopen(URL) as remote, open(JSON, 'wb') as local: #2
            local.write(remote.read())
    
    with open(JSON) as fp:
        return json.load(fp) #3

In [6]:
feed = load() #1

In [7]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [8]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
484 events
357 speakers
 53 venues


In [9]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [10]:
feed['Schedule']['speakers'][-1]['serial']

141590

In [11]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [12]:
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

In [15]:
from collections import abc

class FrozenJson:
    """一个只读接口，使用属性表示法访问JSON类对象"""
    
    def __init__(self, mapping):
        self.__data = dict(mapping) #1
        
    def __getattr__(self, name): #2
        if hasattr(self.__data, name):
            return getattr(self.__data, name) #3
        else:
            return FrozenJson.build(self.__data[name]) #4
    
    @classmethod
    def build(cls, obj): #5
        if isinstance(obj, abc.Mapping): #6
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): #7
            return [cls.build(item) for item in obj]
        else:
            return obj

In [20]:
grad = FrozenJson({'name': 'Jim Bo', 'class': 1982})

In [21]:
grad.class

SyntaxError: invalid syntax (<ipython-input-21-bb5c99ef29c5>, line 1)

In [18]:
getattr(grad, 'class')

1982

In [24]:
from collections import abc
import keyword

class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象"""
    
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
        
    def __getattr__(self, name): #2
        if hasattr(self.__data, name):
            return getattr(self.__data, name) #3
        else:
            return FrozenJson.build(self.__data[name]) #4
    
    @classmethod
    def build(cls, obj): #5
        if isinstance(obj, abc.Mapping): #6
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): #7
            return [cls.build(item) for item in obj]
        else:
            return obj

In [25]:
x = FrozenJson({'2be': 'asdf'})

In [26]:
x.2be

SyntaxError: invalid syntax (<ipython-input-26-8694215ab5bd>, line 1)

In [ ]:
from collections import abc

class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象"""
    
    def __new__(cls, arg): #1
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls) #2
        elif isinstance(arg, abc.MutableSequence): #3
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
            
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJson(self.__data[name]) #4